<a href="https://colab.research.google.com/github/aubricot/computer_vision_with_eol_images/blob/master/classification_for_image_tagging/image_type/cartoonify_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Determine if images are a cartoon or photograph
---
*Last Updated 17 December 2022*   
Classification accuracy for illustrated images and phylogenies was low for the trained model. This notebook uses an alternate approach that leverages image processing to identify images as photographic or non-photographic. First, cartoonify image, then compare change in color values. If change above a certain threshold, then image is likely photographic. If change below a certain threshold, image is likely non-photographic.
  
***Using 500 images from all image type classes, the best predictor of "not cartoon" was found to be Manhattan norm per pixel > 2.***

## Installs & Imports
---

In [ ]:
#@title Choose where to save results & set up directory structure
# Use dropdown menu on right
save = "in Colab runtime (files deleted after each session)" #@param ["in my Google Drive", "in Colab runtime (files deleted after each session)"]
print("Saving results ", save)

# Mount google drive to export file(s)
if 'Google Drive' in save:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

# Type in the path to your working directory in form field to right
import os
basewd = "/content/drive/MyDrive/train/tf2" #@param ["/content/drive/MyDrive/train/tf2"] {allow-input: true}
if not os.path.exists(basewd):
    os.makedirs(basewd)

# Folder where inspect results outputs will be saved
results_folder = "inspect_resul" #@param ["inspect_resul"] {allow-input: true}
cwd = basewd + '/' + results_folder
if not os.path.exists(cwd):
    os.makedirs(cwd)
print("\nWorking directory set to: \n", cwd)

# Enter image classes of interest in form field
filters = ['herb', 'illus', 'map', 'null', 'phylo'] #@param ["['herb', 'illus', 'map', 'null', 'phylo']"] {type:"raw", allow-input: true}

# Folder where image metadata was saved in image_type_preprocessing.ipynb
data_folder = "pre-processing/image_data" #@param ["pre-processing/image_data"] {allow-input: true}
data_wd = basewd + '/' + data_folder
if not os.path.exists(data_wd):
    !pip3 install --upgrade gdown
    os.makedirs(data_wd)
    print("\nDownload image bundles for image type classes {}...\n".format(filters))
    %cd $data_wd
    file_ids = ['1Bkh2-TZSIKCCoKOTNr2L65BwR92Nx0vZ', '1m2sOLpUOWsw5RwzRtvj0mqH8aPloqnE_', \
                '1EIwPxyrawXnTPMyvO8f4nc1e3HALrTp9', '16I-_Qbh2IX_1Oz5wqlE6uzWXB2VhjE3e', \
                '1hQNgRLZWZu77XAxBwQIJOgRmWCCcpMos']
    for file_id in file_ids:
        !gdown $file_id
print("\nImage metadata directory set to: \n", data_wd)

In [ ]:
fns = os.listdir(data_wd)
imclasses = filters
demo_dict = dict(zip(imclasses, fns))

demo_dict = {'herb': 'herbarium_sheets_download.txt', 'illus': ['Botanical_illustrations_download.txt', 'Zoological_illustrations_download.txt'], 'map': 'maps.txt', 'null': None, 'phylo': 'Phylogeny_images.txt'}
print(demo_dict)
print(demo_dict[filters[0]])

In [ ]:
# For importing data and images
import pandas as pd
import numpy as np
from numpy import sum
import os
import scipy
from scipy.linalg import norm
from scipy import average

# For working with images
from PIL import Image
import imageio
import six.moves.urllib as urllib
import cv2
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'svg'

# Define functions

# Define start and stop indices in EOL bundle for running inference   
def set_start_stop(run, df):
    # To test with a tiny subset, use 50 random bundle images
    N = len(df)
    if "tiny subset" in run:
        start=np.random.choice(a=N, size=1)[0]
        stop=start+50
    # To run for a larger set, use 500 random images
    else:
        start=np.random.choice(a=N, size=1)[0]
        stop=start+500
    
    return start, stop

# Make a dictionary of image type classes and corresponding image bundles
demo_dict = {'herb': 'herbarium_sheets_download.txt', 'illus': ['Botanical_illustrations_download.txt', 'Zoological_illustrations_download.txt'], 'map': 'maps.txt', 'null': None, 'phylo': 'Phylogeny_images.txt'}

# Set filename for saving classification results
def get_test_images(imclass):
    impath = cwd + '/pre-processing/images/' + imclass
    # If already custom-trained model, pull test images to inspect results for
    if os.path.exists(impath):
        demo = False # Not running in demo mode
        fns = os.listdir(impath)
        TEST_IMAGE_PATHS = [os.path.join(impath, fn) for fn in fns]
        print("\nUsing test images from: \n", impath)
    # If running this script to test functionality, download dummy dataset from EOL image bundles
    else:
        demo = True # Running in demo mode using only Colab Runtime files
        TEST_IMAGE_PATHS = []
        try:
            fpath = data_wd + '/' + demo_dict[imclass]
            df = pd.read_csv(fpath, sep='\n', header=None)
            start=np.random.choice(a=len(df), size=1)[0]
            stop=start+5
            TEST_IMAGE_PATHS = df.iloc[start:stop, 0].values.tolist()
            print("\nUsing 5 random images from EOL image type bundle: \n", fpath)
        
        except:
            pass

    return TEST_IMAGE_PATHS, demo

# For uploading an image from url
# Modified from https://www.pyimagesearch.com/2015/03/02/convert-url-to-image-with-python-and-opencv/
def url_to_image(url):
    resp = urllib.request.urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    im_h, im_w = image.shape[:2]
 
    return image

# Set filename for saving classification results
def set_outfpath(imclass):
    outfpath = cwd + '/' + imclass + '_cartoonifcation_values.csv'
    print("\nSaving results to: \n", outfpath)

    return outfpath

# To cartoonize an image
def cartoonize(image):
    # Add edges
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
    gray = cv2.medianBlur(gray, 5) 
    edges = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C,  
                                         cv2.THRESH_BINARY, 9, 9)  
    edges = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)
    # Bilateral filter 
    color = cv2.bilateralFilter(img, 9, 250, 250) 
    img2 = cv2.bitwise_and(color, edges)

    return img2

# Calculate differences between original and cartoonized image
def calc_img_diffs(img, img2):
    # Convert both images from RGB to HSV
    HSV_img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    HSV_img2 = cv2.cvtColor(img2, cv2.COLOR_RGB2HSV)
    # Fnd the difference for H of HSV values of the images
    diff = HSV_img[:,:,0]-HSV_img2[:,:,0]
    mnorm = sum(abs(diff))  # Manhattan norm
    mnorm_pp = mnorm/HSV_img.size # per pixel
    znorm = norm(diff.ravel(), 0)  # Zero norm
    znorm_pp = znorm*1.0/HSV_img2.size # per pixel

    return mnorm, mnorm_pp, znorm, znorm_pp

# To display an image already loaded into the runtime
def display_images(image, image2, mnorm, mnorm_pp, znorm, znorm_pp):
    fig, (a,b) = plt.subplots(2, figsize=(5, 5), constrained_layout=True)
    fig.suptitle("Original vs Cartoonized, pairwise differences\nManhattan norm: {} / per pixel: {}\
                  \nZero norm: {} / per pixel: {}".format(mnorm, mnorm_pp, znorm, znorm_pp))
    a.imshow(image) ;
    b.imshow(image2)

# Make placeholder lists to fill for each class
def make_placeholders():
    filenames = []
    mnorms = []
    mnorms_pp = []
    znorms = []
    znorms_pp = []

    return filenames, mnorms, mnorms_pp, znorms, znorms_pp
    
# Add values for each image to placeholder list
def record_results(fn, mnorm, mnorm_pp, znorm, znorm_pp):
    filenames.append(fn)
    mnorms.append(mnorm)
    mnorms_pp.append(mnorm_pp)
    znorms.append(znorm)
    znorms_pp.append(znorm_pp)
    results = [filenames, mnorms, mnorms_pp, znorms, znorms_pp]

    return results

# Export results
def export_results(results):
    results = pd.DataFrame(results)
    results = results.transpose()
    results.to_csv(outfpath, index=False, header=("filename", "mnorm", "mnorm_pp", 
                                                 "znorm", "znorm_pp"))
    
# To save the figure
def save_figure(fig, imclass):
    figname = cwd + '/' + imclass + '_cartoonization_hists.png'
    fig.savefig(figname)
    print("Histograms saved to ", figname)

    return figname

## Cartoonization - Cartoonize images, then compare how different they are to the original and determine if they are likely a cartoon or photograph
---

In [ ]:
#@title Cartoonify images

# Test pipeline with a smaller subset than 5k images?
run = "test with tiny subset" #@param ["test with tiny subset", "for 500 images"]
print("Run: ", run)

# For each image class, measure difference betweeen cartoonified and original
imclasses = filters
for imclass in imclasses:
    # Set filename for saving classification results
    outfpath = set_outfpath(imclass)
    # Make placeholder lists to record values for each image
    filenames, mnorms, mnorms_pp, znorms, znorms_pp = make_placeholders()
    # Get test images for cartoonizing
    df, demo = get_test_images(imclass)

    # Cartoonify images
    try:
        start, stop = set_start_stop(run, df)
        for i, row in enumerate(df[start:stop], start=1):
            # Read in image from url or file
            if demo:
                url = row
                img = url_to_image(url)
            else:
                im_path = row
                img = cv2.imread(im_path)
        
            # Cartoonization
            img2 = cartoonize(img) 

            # Calculate differences between original and cartoonized image
            mnorm, mnorm_pp, znorm, znorm_pp = calc_img_diffs(img, img2)

            # Display cartoonized image when testing with tiny subset
            if "tiny subset" in run:
                display_images(img, img2, mnorm, mnorm_pp, znorm, znorm_pp)

            # Record results in placeholder lists to inspect results in next step
            results = record_results(row, mnorm, mnorm_pp, znorm, znorm_pp)
    except:
        pass

    # Combine to df and export results
    export_results(results)

### Inspect cartoonizaton results
---

In [ ]:
#@title Combine model outputs for image type classes

# Get cartoonization files for each image type class
imclasses = filters
all_filenames = [cwd + '/' + imclass + '_cartoonifcation_values.csv' for imclass in imclasses]
imclass_dict = dict(zip(all_filenames, imclasses))

# Loop through cartoonization files and display histograms
for fn in all_filenames:
    print("\nInspecting cartoonization values for: ", fn)
    imclass = imclass_dict[fn]
    df = pd.read_csv(fn, header=0)
    mnorms = df['mnorm']
    mnorms_pp = df['mnorm_pp']
    znorms = df['znorm']
    znorms_pp = df['znorm_pp']

    # Plot parameters
    kwargs = dict(alpha=0.5, bins=15)
    fig, (a, b, c, d) = plt.subplots(4, figsize=(10, 10), sharey=True, constrained_layout=True)
    fig.suptitle('{}: Image differences after cartoonization (n={} imgs)'.format(imclass, len(df)))

    # Manhattan norm values
    bins, counts = np.histogram(mnorms)
    a.hist(mnorms, color='y', label='True Det', **kwargs)
    a.set_title("{}: Manhattan norm".format(imclass));

    # Zero norm values
    bins, counts = np.histogram(znorms)
    c.hist(znorms, color='y', label='True Det', **kwargs)
    c.set_title("{}: Zero norm".format(imclass));

    # Manhattan norm values per pixel
    bins, counts = np.histogram(mnorms_pp)
    b.hist(mnorms_pp, color='y', label='True Det', **kwargs)
    b.set_title("{}: Manhattan norm per pixel".format(imclass));

    # Zero norm values per pixel
    bins, counts = np.histogram(znorms_pp)
    d.hist(znorms_pp, color='y', label='True Det', **kwargs)
    d.set_title("{}: Zero norm per pixel".format(imclass));

    # Export histograms
    figname = save_figure(fig, imclass)